In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import csv
from pathlib import Path

traits = [
    "adventerous","challenging","dark","emotional","funny","hopeful",
    "informative","inspiring","lighthearted","mysterious","reflective",
    "relaxing","sad","tense"
]

trait_sliders = {
    t: widgets.FloatSlider(
        value=0.0,
        min=0.0,
        max=1.0,
        step=0.01,
        description=t,
        continuous_update=True,
        readout_format=".2f",
        layout=widgets.Layout(width="320px")
    )
    for t in traits
}

grid = widgets.GridBox(
    children=list(trait_sliders.values()),
    layout=widgets.Layout(
        grid_template_columns="repeat(2, 340px)",
        grid_gap="4px 12px"
    )
)

author_input = widgets.Text(
    value="none",
    description="Author",
    placeholder="Enter author name",
    layout=widgets.Layout(width="320px")
)

btn_author_none = widgets.Button(description="None", tooltip="Clear author filter")

btn_short  = widgets.Button(description="Short",  button_style="success")
btn_medium = widgets.Button(description="Medium", button_style="info")
btn_long   = widgets.Button(description="Long",   button_style="warning")

# Clear borders initially
for b in (btn_short, btn_medium, btn_long):
    b.layout.border = 'none'

output_area = widgets.Output()

selected_length = widgets.Text(
    value="short",
    description="Chosen",
    disabled=True,
    layout=widgets.Layout(width="200px")
)

book_label = widgets.HTML("<b> Select a book preset to autofill preferences</b>")
author_label = widgets.HTML("<b> Select preferred author</b>")
length_label = widgets.HTML("<b> Select preferred length</b>")
mood_label = widgets.HTML("<b> Select preferred moods</b>")
pace_label = widgets.HTML("<b> Select preferred pace</b>")

btn_pace_fast   = widgets.Button(description="Fast",   button_style="success")
btn_pace_medium = widgets.Button(description="Medium", button_style="info")
btn_pace_slow   = widgets.Button(description="Slow",   button_style="warning")

selected_pace = widgets.Text(
    value="fast",
    description="Pace",
    disabled=True,
    layout=widgets.Layout(width="200px")
)

def set_length(choice):
    selected_length.value = choice
    for b in (btn_short, btn_medium, btn_long):
        b.layout.border = 'none'
    # Add border to selected
    if choice == "short":
        btn_short.layout.border = "3px solid #2e7d32"   # green
    elif choice == "medium":
        btn_medium.layout.border = "3px solid #0288d1"  # blue
    elif choice == "long":
        btn_long.layout.border = "3px solid #f57c00"    # orange

def clear_author(_):
    author_input.value = "none"

def set_pace(choice):
    selected_pace.value = choice
    for b in (btn_pace_fast, btn_pace_medium, btn_pace_slow):
        b.layout.border = 'none'
    # Add border to selected
    if choice == "fast":
        btn_pace_fast.layout.border = "3px solid #2e7d32"   # green
    elif choice == "medium":
        btn_pace_medium.layout.border = "3px solid #0288d1"  # blue
    elif choice == "slow":
        btn_pace_slow.layout.border = "3px solid #f57c00"    # orange

btn_author_none.on_click(clear_author)
btn_short.on_click(lambda _: set_length("short"))
btn_medium.on_click(lambda _: set_length("medium"))
btn_long.on_click(lambda _: set_length("long"))

btn_pace_fast.on_click(lambda _: set_pace("fast"))
btn_pace_medium.on_click(lambda _: set_pace("medium"))
btn_pace_slow.on_click(lambda _: set_pace("slow"))


BOOK_CSV_PATH = Path("book_list.csv")

def classify_length_from_pages(pages: int) -> str:
    if pages <= 250:
        return "short"
    if pages <= 500:
        return "medium"
    return "long"

trait_keys = [
    "adventerous","challenging","dark","emotional","funny","hopeful",
    "informative","inspiring","lighthearted","mysterious","reflective",
    "relaxing","sad","tense"
]

def load_book_rows(path: Path):
    if not path.exists():
        return []
    rows = []
    with path.open(newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append({k.lower().strip(): v for k, v in r.items()})
    return rows

CHOSEN_TITLES = [
    "The Name of the Wind",
    "The Hobbit",
    "The Night Circus",
    "Words of Radiance",
    "Furies of Calderon",
    "Mort",
    "The Spellshop",
    "Red Sister",
    "Babel",
    "Magician",
]

all_rows = load_book_rows(BOOK_CSV_PATH)

def select_rows_by_titles(rows, wanted_titles):
    if not wanted_titles:
        return rows[:10]

    by_title = { (r.get("title","").strip().lower()): r for r in rows }
    selected = []
    missing = []
    for t in wanted_titles[:10]:
        key = t.strip().lower()
        r = by_title.get(key)
        if r:
            selected.append(r)
        else:
            missing.append(t)
    if missing and output_area:
        with output_area:
            print("Warning - titles not found in CSV:", ", ".join(missing))
    return selected

raw_books = select_rows_by_titles(all_rows, CHOSEN_TITLES)

book_buttons = []
book_presets = []

for r in raw_books:
    title = r.get("title", "").strip()
    if not title:
        continue
    author_val = r.get("author", "").strip()
    length_field = r.get("length", "").strip()
    length_cat = ""
    if length_field.isdigit():
        length_cat = classify_length_from_pages(int(length_field))
    else:
        lf = length_field.lower()
        if lf in ("short","medium","long"):
            length_cat = lf
        else:
            length_cat = "medium"  # fallback
    pace_val = r.get("pace", "").strip().lower()
    if pace_val not in ("fast","medium","slow"):
        pace_val = "medium"

    mood_values = []
    for mk in trait_keys:
        raw_val = r.get(mk, "").strip()
        try:
            mv = float(raw_val)/100
        except:
            mv = 0.0
        if mv < 0: mv = 0.0
        if mv > 1: mv = 1.0
        mood_values.append(mv)

    book_presets.append({
        "title": title,
        "author": author_val,
        "length": length_cat,
        "pace": pace_val,
        "moods": {k: v for k, v in zip(trait_keys, mood_values)}
    })

# Create buttons (show title, truncate if long)
for bp in book_presets:
    label = bp["title"][:24] + ("…" if len(bp["title"]) > 24 else "")
    b = widgets.Button(description=label, layout=widgets.Layout(width="180px"))
    book_buttons.append(b)

books_box = widgets.HBox(book_buttons, layout=widgets.Layout(flex_flow="row wrap"))

def apply_book_preset(preset):
    set_length(preset["length"])
    set_pace(preset["pace"])
    for mk, val in preset["moods"].items():
        trait_sliders[mk].value = val

for btn, preset in zip(book_buttons, book_presets):
    btn.on_click(lambda _, p=preset: apply_book_preset(p))


ui = widgets.VBox([
    book_label,
    books_box,
    author_label,
    widgets.HBox([author_input, btn_author_none]),
    length_label,
    widgets.HBox([btn_short, btn_medium, btn_long]),
    selected_length,
    pace_label,
    widgets.HBox([btn_pace_fast, btn_pace_medium, btn_pace_slow]),
    selected_pace,
    mood_label,
    grid,
    output_area
])

display(ui)

def get_current_preferences():
    # Return: [title(always "none"), author, length, [mood values], pace]
    title = "none"
    author = ((author_input.value or "").strip() or "none")
    length = selected_length.value.strip()
    mood_values = [trait_sliders[t].value for t in traits]
    pace = selected_pace.value.strip()
    return [title, author, length, mood_values, pace]

def on_any_change(change):
    if change["name"] == "value":
        pass

for s in trait_sliders.values():
    s.observe(on_any_change)

In [23]:
print(get_current_preferences())

['none', 'none', 'medium', [0.42, 0.01, 0.0, 0.25, 0.44, 0.64, 0.0, 0.23, 0.87, 0.07, 0.07, 0.43, 0.01, 0.02], 'medium']
